<a href="https://colab.research.google.com/github/yingzhi-w/text_mining/blob/main/LDA_Topic_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyLDAvis=="2.1.2"
!pip install NLTK
!pip install Gensim

     |████████████████████████████████| 1.6 MB 5.1 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=105d0712391af43f5a7015293c26c1a3f828e64855b6ef9e5c547776b3cf2c89
  Stored in directory: /root/.cache/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis
  Attempting uninstall: pyLDAvis
    Found existing installation: pyLDAvis 3.3.1
    Uninstalling pyLDAvis-3.3.1:
      Successfully uninstalled pyLDAvis-3.3.1


In [ ]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [ ]:
stopwords = stopwords.words("english")

In [ ]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

data = load_data("gdrive/My Drive/pyLDA/ushmm_dn.json")["texts"]

print (data[0][0:90])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928. 


In [ ]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

name bear small town call bear very hard work child father mother small mill flour buckwhe


In [ ]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [ ]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 10), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 2), (8, 3), (9, 1), (10, 12), (11, 1), (12, 8), (13, 1), (14, 2), (15, 1), (16, 3), (17, 2), (18, 1), (19, 2)]
able


In [ ]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 10), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 2), (8, 3), (9, 1), (10, 12), (11, 1), (12, 8), (13, 1), (14, 2), (15, 1), (16, 3), (17, 2), (18, 1), (19, 2)]
able


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return pd.DataFrame({'Term': vocab[term_ix], \


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.299008  0.056220       1        1  39.901776
26     0.408927  0.063305       2        1  27.767818
19     0.363364  0.093258       3        1  10.229733
21     0.448391  0.105327       4        1   9.313238
5      0.327235 -0.040077       5        1   9.020560
0      0.435809 -0.136743       6        1   2.032341
10     0.272596  0.265089       7        1   0.867951
6      0.333202 -0.215628       8        1   0.685857
17     0.016175 -0.275328       9        1   0.080222
11    -0.207997 -0.261526      10        1   0.054160
8     -0.171583  0.167276      11        1   0.036565
4     -0.137797  0.000680      12        1   0.004218
18    -0.133985  0.009607      13        1   0.000896
1     -0.132539  0.010168      14        1   0.000398
23    -0.132537  0.009898      15        1   0.000285
25    -0.132537  0.009898      16        1   0.000284
13    -0.132537  0.009898      17        1   0.000283
28    -0.132546  0.009898      18        1   0.000274
16    -0.132552  0.009898      19        1   0.000269
14    -0.132551  0.009898      20        1   0.000269
27    -0.132552  0.009898      21        1   0.000267
15    -0.132552  0.009898      22        1   0.000267
29    -0.132552  0.009898      23        1   0.000265
7     -0.132553  0.009898      24        1   0.000264
2     -0.132554  0.009898      25        1   0.000264
12    -0.132554  0.009898      26        1   0.000263
9     -0.132555  0.009898      27        1   0.000258
20    -0.132557  0.009898      28        1   0.000253
24    -0.132558  0.009898      29        1   0.000253
22    -0.132559  0.009898      30        1   0.000251, topic_info=       Term          Freq         Total Category  logprob  loglift
751      so  20550.000000  20550.000000  Default  30.0000  30.0000
867    very  11216.000000  11216.000000  Default  29.0000  29.0000
350      go  24069.000000  24069.000000  Default  28.0000  28.0000
601  people  12578.000000  12578.000000  Default  27.0000  27.0000
170    come  14045.000000  14045.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
822   think      0.000161   5446.380804  Topic30  -9.6310  -4.4375
819    then      0.000161   7974.200347  Topic30  -9.6310  -4.8188
67     back      0.000161   5057.133915  Topic30  -9.6310  -4.3634
486    look      0.000161   3446.837643  Topic30  -9.6311  -3.9801
133     can      0.000161   5030.892393  Topic30  -9.6311  -4.3583

[2043 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
14186     11  1.023888   abet
0          1  0.580656   able
0          2  0.100447   able
0          3  0.200895   able
0          4  0.002643   able
...      ...       ...    ...
919        4  0.097171  young
919        5  0.008260  young
919        6  0.017977  young
919        7  0.009717  young
919        8  0.033524  young

[2224 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 27, 20, 22, 6, 1, 11, 7, 18, 12, 9, 5, 19, 2, 24, 26, 14, 29, 17, 15, 28, 16, 30, 8, 3, 13, 10, 21, 25, 23])